El presente código tiene por objetivo ejecutar el preprocesamiento de datos meteorológicos.

Luego de importar la base de datos original, se eliminan las variables que no se considerarán en el proyecto; a continuación, se agregan filas correspondientes a los días no registrados en la base de datos. Se agregan columnas correspondientes a la latitud, longitud y altitud de las estaciones, así como una columna que indica el número de mes de cada registro.

Se realiza un proceso de rellenado de datos faltantes. El proceso seguido para las diferentes variables meteorológicas es:

1. Precipitación, temperatura máxima, media y mínima: Se sustituyen valores desconocidos por los proporcionados por la base de datos de ENACTS; los datos de ENACTS son importados de archivos en formato NetCDF con nombre 'precipitacion.nc', 'tmax.nc', 'tmed.nc', 'tmin.nc'.

2. Humedad relativa, velocidad del viento y dirección del viento: Se realiza, para cada día desde el 1/1/1980 hasta el 31/5/2023, una interpolación geográfica utilizando la latitud y la longitud de las estaciones cuyo valor es conocido. Se utiliza Kriging a partir del paquete PyKrige; el método utilizado es interpolación lineal. Se evalúa, en la interpolación, la latitud y longitud de las estaciones con valor desconocido para hallar el valor a utilizar.

3. Presión atmosférica: Se realiza, para cada día desde el 1/1/1980 hasta el 31/5/2023, una regresión lineal de presión atmosférica vs altitud con las cantidades de las estaciones de valor conocido. Se utiliza el paquete Scikit Learn para realizar la regresión. Se evalúa la altitud de las estaciones con valor desconocido para hallar el valor a utilizar.

Seguidamente, se crea un dataframe con el listado de todos los días del año para cada una de las estaciones, que contiene la media, la desviación estándar, así como los percentiles P(1/3) y P(2/3) del conjunto de valores de precipitación de la Climatología. Este dataframe se exporta en un Excel hacia los archivos del usuario con nombre 'Percentiles.xlsx'. A partir de este dataframe, se agregan a la base de datos meteorológicos las variables objetivo:

1. Anomalía absoluta de precipitación del día siguiente, respecto de la Climatología. (Variable continua)
2. Clasificación de precipitación del día siguiente como =0 ó >0 (Variable categórica)
3. Nivel cualitativo de precipitación del día siguiente en términos de los percentiles de la Climatología. (Variable categórica)

Con respecto a la segunda variable:

- Se ingresa 0 si la precipitación es =0
- Se ingresa 1 si la precipitación es >0

Con respecto a la tercera variable:

- Se ingresa -1 si la precipitación es inferior o igual al percentil P(1/3).
- Se ingresa +1 si la precipitación es superior o igual al percentil P(2/3).
- Se ingresa 0 en otro caso.

A continuación, se agrega columnas con las mediciones de las cantidades meteorológicas para cada uno de los últimos 7 días.

Finalmente, se seleccionan las columnas que se desean en la base de datos final, y se exporta la misma hacia un archivo llamado 'BaseDatosProcesada.xlsx'

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# IMPORTACIÓN DE DATOS

In [ ]:
# Se carga la base de datos original

import pandas as pd #Para manejo de dataframes
import numpy as np #Para análisis de datos

file_path = 'BaseDatos.xlsx'
Datos = pd.read_excel(file_path)

print(Datos)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ELIMINACIÓN DE VARIABLES A DESCARTAR

In [ ]:
VarEliminar = ['Radiacion','Brillo_Solar','Evap_Tanque','Piche'] #Lista de variables a eliminar

Datos.drop(columns=VarEliminar, inplace=True) #Eliminamos las columnas

print(Datos)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# AGREGAR DÍAS NO REGISTRADOS

EN PRIMER LUGAR, SE DEBE GENERAR UN REGISTRO DE LOS DÍAS QUE NO ESTÁN INGRESADOS, PARA CADA UNA DE LAS ESTACIONES

In [ ]:
# Crear un DataFrame que contenga todas las fechas posibles en el rango de fechas para cada estación
start_date = Datos['FECHA'].min()
end_date = Datos['FECHA'].max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Crear una lista para almacenar los resultados
missing_dates = []

# Iterar sobre cada estación
for estacion in Datos['ESTACIÓN'].unique():
    # Filtrar los datos por estación
    Datos_estacion = Datos[Datos['ESTACIÓN'] == estacion]
    
    # Obtener las fechas presentes en la base de datos para esta estación
    fechas_present = Datos_estacion['FECHA']
    
    # Obtener las fechas faltantes para esta estación
    fechas_missing = all_dates.difference(fechas_present)
    
    # Agregar las fechas faltantes a la lista de resultados
    for date in fechas_missing:
        missing_dates.append({'ESTACIÓN': estacion, 'FECHA': date})

# Convertir la lista de resultados a un DataFrame
missing_dates_Datos = pd.DataFrame(missing_dates)

print("Las fechas no ingresadas en la base de datos son: ")
print(missing_dates_Datos)

A CONTINUACIÓN, SE AGREGAN A LA BASE DE DATOS LOS DÍAS NO INGRESADOS; TODOS LOS VALORES DE ESTOS DÍAS APARECERAN VACÍOS

In [ ]:
# Añadir las columnas de datos meteorológicos con valores NaN
for col in Datos.columns:
    if col not in ['ESTACIÓN', 'FECHA']:
        missing_dates_Datos[col] = np.nan

# Combinar el DataFrame original con el DataFrame de fechas faltantes
Datos = pd.concat([Datos, missing_dates_Datos], ignore_index=True)

# Ordenar el DataFrame resultante por estación y fecha
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

print(Datos)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# AGRAGAR LAT, LON, ALT, DÍA DEL MES

A CONTINUACIÓN, SE AGREGA LA LATITUD, LONGITUD Y ALTURA DE LA ESTACIÓN, ASÍ COMO EL NÚMERO DE MES

In [ ]:
# Importar la base de datos que contiene los nombres de las estaciones y su latitud, longitud y altura.
file_path = 'Datos_estaciones.xlsx'
Datos_estaciones = pd.read_excel(file_path)

# Fusionar la base de datos con los datos de las estaciones, en función del nombre de la estación
Datos = pd.merge(Datos, Datos_estaciones, on='ESTACIÓN', how='left')

# Agregar una nueva columna con el número del mes
Datos['MES'] = Datos['FECHA'].dt.month

print(Datos)

In [ ]:
# Guardamos el avance en un dataframe
Preliminar1 = Datos

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# RELLENO DE DATOS FALTANTES

### LLUVIA, TMAX, TMED, TMIN

EN PRIMER LUGAR, SE RELIZA EL RELLENO DE DATOS FALTANTES DE LLUVIA Y TEMPERATURA MEDIA, MÁXIMA Y MÍNIMA, POR MEDIO DE LOS DATOS DE ENACTS

In [ ]:
# Verificar cuántos valores son desconocidos
print("Valores vaciós de precipitación: " + str(Datos['Lluvia'].isna().sum()))
print("Valores vaciós de temperatura media: " + str(Datos['Tmed'].isna().sum()))
print("Valores vaciós de temperatura máxima: " + str(Datos['Tmax'].isna().sum()))
print("Valores vaciós de temperatura mínima: " + str(Datos['Tmin'].isna().sum()))

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import warnings

# Suprimir advertencias de tipo FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Crear una copia del DataFrame original, la cual se irá rellenando
Datos_rellenados = Datos.copy()

# Mapeo de variables de la base de datos a las variables en los archivos NetCDF (esto se hace ya que el nombre de la variable
# en la base de datos meteorológicos puede no tener el mismo nombre que la variable en el archivo NetCDF)
var_map = {
    'Lluvia': 'rfe',
    'Tmin': 'tmin',
    'Tmax': 'tmax',
    'Tmed': 'tmed'
}

# Archivos NetCDF correspondientes
nc_files = {
    'rfe': 'precipitacion.nc',
    'tmin': 'tmin.nc',
    'tmax': 'tmax.nc',
    'tmed': 'tmed.nc'
}

# Cargar los archivos NetCDF en un diccionario
nc_data = {var: xr.open_dataset(file) for var, file in nc_files.items()}

# Iterar sobre las filas
for i, row in Datos.iterrows():
    # Extraer latitud, longitud y fecha para la fila actual
    lat = row['Latitud']
    lon = row['Longitud']
    fecha = row['FECHA']
    estacion = row['ESTACIÓN']
    
    # Iterar sobre las variables para la fila actual
    for db_var, nc_var in var_map.items():
        if pd.isna(row[db_var]):
            # Extraer los datos de la variable correspondiente del archivo NetCDF
            valor = nc_data[nc_var][nc_var].sel(
                Y=lat, X=lon, T=fecha, method='nearest'
            ).values
            
            # Reemplazar el valor en el DataFrame
            Datos_rellenados.at[i, db_var] = valor
            
    print("Se han rellenado las variables en la estación " + "'" + estacion + "'" + " en la fecha " + str(fecha))

print(Datos_rellenados)

In [ ]:
# Actualizar el DataFrame original con los datos rellenados
Datos = Datos_rellenados

print(Datos)

In [ ]:
# Verificar que no quede ningún valor vacío
print("Valores vaciós de precipitación: " + str(Datos['Lluvia'].isna().sum()))
print("Valores vaciós de temperatura media: " + str(Datos['Tmed'].isna().sum()))
print("Valores vaciós de temperatura máxima: " + str(Datos['Tmax'].isna().sum()))
print("Valores vaciós de temperatura mínima: " + str(Datos['Tmin'].isna().sum()))

### HUMEDAD_R, VEL_VIENTO, DIR_VIENTO

A CONTINUACIÓN, SE RELLENAN DATOS FALTANTES DE HUMEDAD, VELOCIDAD Y DIRECCIÓN DEL VIENTO MEDIANTE UNA INTERPOLACIÓN GEOMÉTRICA PARA CADA DÍA, UTILIZANDO LA METODOLOGÍA 'KRIGING'

In [ ]:
# Verificar cuántos valores son desconocidos
print("Valores vaciós de humedad relativa: " + str(Datos['Humedad_R'].isna().sum()))
print("Valores vaciós de velocidad del viento: " + str(Datos['Vel_viento'].isna().sum()))
print("Valores vacíos de dirección del viento: " + str(Datos['Dir_viento'].isna().sum()))

In [ ]:
from pykrige.ok import OrdinaryKriging
import pandas as pd
import numpy as np

# Lista de variables a procesar mediante interpolación geográfica
variables = ['Humedad_R', 'Vel_viento', 'Dir_viento']

# Inicializa un DataFrame para almacenar los resultados
Datos_rellenados = pd.DataFrame()

# Obtiene la lista de fechas únicas
fechas_unicas = Datos['FECHA'].unique()

for fecha in fechas_unicas:
    # Filtra los datos para la fecha actual
    df_dia = Datos[Datos['FECHA'] == fecha]
    
    for variable in variables:
        
        # Agregar un mensaje que permita conocer el avance del proceso
        print("Se ha rellenado la variable " + "'" + variable + "' " + "en la fecha " + str(fecha))
        
        # Filtra los datos conocidos (no nulos) para la variable actual
        known_data = df_dia.dropna(subset=[variable])
        
        # Si no hay datos conocidos, salta la interpolación
        if known_data.empty:
            continue
        
        # Extrae las coordenadas y valores conocidos
        known_coords = known_data[['Latitud', 'Longitud']].values
        known_var = known_data[variable].values
        
        # Verifica si todos los valores de la variable conocida son cero
        if np.all(known_var == 0):
            df_dia.loc[df_dia[variable].isnull(), variable] = 0
            continue
        
        # Filtra los datos desconocidos (nulos)
        unknown_data = df_dia[df_dia[variable].isnull()]
        
        # Si no hay datos desconocidos, no se necesita interpolación
        if unknown_data.empty:
            continue
        
        # Extrae las coordenadas desconocidas
        unknown_coords = unknown_data[['Latitud', 'Longitud']].values
        
        # Realiza la interpolación utilizando Ordinary Kriging
        OK = OrdinaryKriging(
            known_coords[:, 1], known_coords[:, 0], known_var,
            variogram_model='linear', verbose=False, enable_plotting=False
        )
        
        # Interpola los valores para las coordenadas desconocidas
        interpolated_var, ss = OK.execute('points', unknown_coords[:, 1], unknown_coords[:, 0])
        
        #Reemplazar posibles valores negativos por cero
        interpolated_var[interpolated_var < 0] = 0
        
        # Reemplaza los valores nulos en el DataFrame original
        df_dia.loc[df_dia[variable].isnull(), variable] = interpolated_var
    
    # Concatenar los resultados al DataFrame resultante
    Datos_rellenados = pd.concat([Datos_rellenados, df_dia])

print(Datos_rellenados)

In [ ]:
# Actualizar el DataFrame original con los datos rellenados
Datos = Datos_rellenados

print(Datos)

In [ ]:
# Verificar que no quede ningún valor vacío
print("Valores vaciós de humedad relativa: " + str(Datos['Humedad_R'].isna().sum()))
print("Valores vaciós de velocidad del viento: " + str(Datos['Vel_viento'].isna().sum()))
print("Valores vacíos de dirección del viento: " + str(Datos['Dir_viento'].isna().sum()))

### PRESION_ATMS

AHORA, SE RELIZA EL RELLENO DE DATOS FALTANTES DE PRESIÓN ATMOSFÉRICA, A TRAVÉS DE UNA REGRESIÓN LINEAL CON LOS VALORES DE PRESIÓN VS ALTURA PARA LAS ESTACIONES CON VALOR CONOCIDO.

In [ ]:
# Verificar cuántos valores son desconocidos
print("Valores vaciós de presión atmosférica: " + str(Datos['Presion_atms'].isna().sum()))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Lista de variables de temperatura a procesar
variables = ['Presion_atms']

# Inicializa un DataFrame para almacenar los resultados
Datos_rellenados = pd.DataFrame()

# Obtiene la lista de fechas únicas
fechas_unicas = Datos['FECHA'].unique()

for fecha in fechas_unicas:
    # Filtra los datos para la fecha actual
    df_dia = Datos[Datos['FECHA'] == fecha]
    
    for variable in variables:
        
        # Agregar un mensaje que permita conocer el avance del proceso
        print("Se ha rellenado la variable " + "'" + variable + "' " + "en la fecha " + str(fecha))
        
        # Filtra los datos conocidos (no nulos) para la variable actual
        known_data = df_dia.dropna(subset=[variable])
        
        # Si no hay datos conocidos, salta la interpolación
        if known_data.empty:
            continue
        
        # Extrae las coordenadas de altitud y valores conocidos
        known_altitude = known_data[['Altitud']].values
        known_atm = known_data[variable].values
        
        # Filtra los datos desconocidos (nulos)
        unknown_data = df_dia[df_dia[variable].isnull()]
        
        # Si no hay datos desconocidos, no se necesita interpolación
        if unknown_data.empty:
            continue
        
        # Extrae las altitudes desconocidas
        unknown_altitude = unknown_data[['Altitud']].values
        
        # Realiza la regresión lineal para la variable actual
        model = LinearRegression()
        model.fit(known_altitude, known_atm)
        
        # Predice los valores para las altitudes desconocidas
        predicted_atm = model.predict(unknown_altitude)
        
        # Reemplazar posibles valores negativos por cero
        predicted_atm[predicted_atm < 0] = 0
        
        # Reemplaza los valores nulos en el DataFrame original
        df_dia.loc[df_dia[variable].isnull(), variable] = predicted_atm
    
    # Concatenar los resultados al DataFrame resultante
    Datos_rellenados = pd.concat([Datos_rellenados, df_dia])

print(Datos_rellenados)

In [ ]:
# Actualizar el DataFrame original con los datos rellenados
Datos = Datos_rellenados

print(Datos)

In [ ]:
# Verificar cuántos valores quedan vacíos
print("Valores vaciós de presión atmosférica: " + str(Datos['Presion_atms'].isna().sum()))

In [ ]:
# Finalmente, guardamos el avance en un dataframe
Preliminar2 = Datos

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# AGREGAR VARIABLE OBJETIVO

In [ ]:
# Ordenar los datos por fecha y estación
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

# Crear una columna de "día del año" para posteriormente poder agrupar los datos por día
Datos['DiaDelAno'] = Datos['FECHA'].dt.strftime('%m-%d')

# Filtrar los datos de la Climatología (1991 - 2020)
Datos_Climatologia = Datos[(Datos['FECHA'].dt.year >= 1991) & (Datos['FECHA'].dt.year <= 2020)]

### DATOS CLIMATOLOGÍA

EN PRIMER LUGAR, SE GENERA UN DATAFRAME CON EL LISTADO DE TODAS LAS ESTACIONES, PARA CADA DÍA DEL AÑO (366), INDICANDO LA MEDIA Y DESVIACIÓN ESTANDAR DEL CONJUNTO DE PRECIPITACIONES DE LA CLIMATOLOGÍA PARA DICHO DÍA, ASÍ COMO EL VALOR DEL PERCENTIL 1/3 Y EL PERCENTIL 2/3

In [ ]:
# Crear un dataframe para almacenar los percentiles
percentiles_df = pd.DataFrame(columns=['ESTACIÓN', 'DiaDelAno', 'Media','Desv_est', 'Percentil_1_/3', 'Percentil_2/3'])

# Obtener las estaciones únicas
estaciones_unicas = Datos['ESTACIÓN'].unique()

# Calcular los percentiles 1/3 y 2/3 para cada estación y día del año
for estacion in estaciones_unicas:
    for dia in Datos['DiaDelAno'].unique():  # Iterar sobre cada día del año en formato mm-dd
        # Filtrar los datos por estación y día del año
        datos_dia_estacion = Datos_Climatologia[(Datos_Climatologia['ESTACIÓN'] == estacion) & 
                                             (Datos_Climatologia['DiaDelAno'] == dia)]['Lluvia']
        
        # Calcular los percentiles
        percentil_1_3 = np.percentile(datos_dia_estacion, 100/3)
        percentil_2_3 = np.percentile(datos_dia_estacion, 200/3)
        promedio = datos_dia_estacion.mean()
        desviacion_estandar = datos_dia_estacion.std()

        # Agregar los resultados al dataframe
        percentiles_df = percentiles_df.append({'ESTACIÓN': estacion,
                                                'DiaDelAno': dia,
                                                'Media': promedio,
                                                'Desv_est': desviacion_estandar,
                                                'Percentil_1_/3': percentil_1_3,
                                                'Percentil_2/3': percentil_2_3},
                                                ignore_index=True)
        
        # Agregar mensaje que indique el avance del proceso
        print("Se ha realizado el proceso para la estación " + estacion + " para el día " + dia)

# Guardar el DataFrame con los percentiles de cada día para la Climatología un nuevo archivo Excel
output_file_path = 'Percentiles.xlsx'
percentiles_df.to_excel(output_file_path, index=False)

print("El proceso ha culminado, los resultados han sido exportados en un archivo con nombre: " + output_file_path)

# Mostrar el dataframe con los percentiles calculados
print(percentiles_df)

### VARIABLE OBJETIVO 1 (CONTINUA) (PARA MODELOS DE REGRESIÓN)

SE AGREGA UNA COLUMNA A LA BASE DE DATOS CON LA PRIMERA VARIABLE OBJETIVO: LA ANOMALÍA ABSOLUTA DE PRECIPITACIÓN DEL DÍA SIGUIENTE, RESPECTO DE LA CLIMATOLOGÍA

In [ ]:
# Hacemos un merge de los datos meteorológicos con los datos de la Climatología
Datos = Datos.merge(percentiles_df, on=['ESTACIÓN', 'DiaDelAno'], how='left')

# Creamos la nueva columna de anomalía absoluta
Datos['Anom_prec_mañana'] = Datos['Lluvia'] - Datos['Media']

# Eliminar las columnas "Desv_est", "Percentil_1_/3" y "Percentil_2/3" ya que no son necesaria en la base de datos meteorológicos
Datos = Datos.drop(columns=['Desv_est','Percentil_1_/3','Percentil_2/3'])

# Ordenar los datos por fecha y estación
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

# Desplazamos la columna 'ANOMALIA_PRECIPITACION' una fila hacia atrás ya que queremos el dato del día siguiente
Datos['Anom_prec_mañana'] = Datos.groupby('ESTACIÓN')['Anom_prec_mañana'].shift(-1)

# Desplazamos la columna "Media" una fila hacia atrás ya que queremos la media de la precipitación del día siguiente
Datos['Media'] = Datos.groupby('ESTACIÓN')['Media'].shift(-1)

# Renombramos la columna "Media" por "Media_prec_mañana_Clim"
Datos.rename(columns={'Media': 'Media_prec_mañana_Clim'}, inplace=True)

print(Datos)

### VARIABLE OBJETIVO 2 (CATEGÓRICA) (PARA MODELOS DE CLASIFICACIÓN) (LLUEVE SÍ/NO)

SE AGREGA UNA COLUMNA CON LA VARIABLE OBJETIVO: CLASIFICACIÓN DE PRECIPITACIÓN DEL DÍA SIGUIENTE COMO =0 (NO LLUEVE) ó >0 (SÍ LLUEVE)

Si la lluvia del dia siguiente, en comparación con la Climatología, es:
- $=0mm$, se ingresa 0 
- $>0mm$, se ingresa +1

In [ ]:
# Creamos una función que nos devuelva 0 si no llueve y 1 si sí llueve
def Lluvia_Binaria(lluvia):
    if lluvia == 0:
        return 0
    else:
        return 1

# Creamos la nueva columna de clasificación binaria de la lluvia usando la función previamente definida
Datos['Lluvia_Binaria'] = Datos.apply(
    lambda row: Lluvia_Binaria(row['Lluvia']), 
    axis=1
)

# Ordenar los datos por fecha y estación
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

# Desplazar la columna una fila hacia atrás ya que queremos el dato del día de mañana
Datos['Lluvia_Binaria'] = Datos.groupby('ESTACIÓN')['Lluvia_Binaria'].shift(-1)

print(Datos)

### VARIABLE OBJETIVO 3 (CATEGÓRICA) (PARA MODELOS DE CLASIFICACIÓN) (COTAS PERCENTILES)

SE AGREGA UNA COLUMNA CON LA VARIABLE OBJETIVO: NIVEL CUALITATIVO DE LA PRECIPITACIÓN DEL DÍA SIGUIENTE EN TÉRMINOS DE LOS PERCENTILES DE LA CLIMATOLOGÍA

Si la lluvia del dia siguiente, en comparación con la Climatología, es:
- Inferior o igual al P(1/3), se ingresa -1 
- Superior o igual al P(2/3), se ingresa +1
- Mayor al P(1/3) y menor al P(2/3), se intresa 0

In [ ]:
# Hacemos un merge de los datos meteorológicos con los datos de la Climatología
Datos = Datos.merge(percentiles_df, on=['ESTACIÓN', 'DiaDelAno'], how='left')

# Crear la columna que indique el percentil en el que se encuentra la lluvia del día siguiente
def percentil(precipitacion, Percentil_un_tercio, Percentil_dos_tercios):
    if Percentil_un_tercio == 0 and Percentil_dos_tercios == 0 and precipitacion == 0:
        return 0
    else:
        if precipitacion <= Percentil_un_tercio:
            return -1
        elif precipitacion >= Percentil_dos_tercios:
            return 1
        else:
            return 0

Datos['percentil'] = Datos.apply(
    lambda row: percentil(row['Lluvia'], row['Percentil_1_/3'], row['Percentil_2/3']), 
    axis=1
)

# Ordenar los datos por fecha y estación
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

# Desplazar la columna de comportamiento cualitativo una fila hacia atrás ya que queremos el dato del día de mañana
Datos['percentil'] = Datos.groupby('ESTACIÓN')['percentil'].shift(-1)

# Eliminar las columnas "media", "Desv_est", "Percentil_1_/3" y "Percentil_2/3" ya que no son necesaria en la base de datos meteorológicos
Datos = Datos.drop(columns=['Media','Desv_est','Percentil_1_/3','Percentil_2/3'])

print(Datos)

In [ ]:
# Finalmente, guardamos el avance en un dataframe
Preliminar3 = Datos

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# GENERAR UN DATAFRAME CON LOS VALORES DE LOS ÚLTIMOS 7 DÍAS

In [ ]:
# Ordenar los datos por fecha y estación
Datos = Datos.sort_values(by=['ESTACIÓN', 'FECHA'])

# Crear nuevas columnas para las variables de los 7 días anteriores
for i in range(1, 8):
    Datos[f'Lluvia_{i}d'] = Datos.groupby('ESTACIÓN')['Lluvia'].shift(i)
    Datos[f'Humedad_R_{i}d'] = Datos.groupby('ESTACIÓN')['Humedad_R'].shift(i)
    Datos[f'Tmin_{i}d'] = Datos.groupby('ESTACIÓN')['Tmin'].shift(i)
    Datos[f'Tmax_{i}d'] = Datos.groupby('ESTACIÓN')['Tmax'].shift(i)
    Datos[f'Tmed_{i}d'] = Datos.groupby('ESTACIÓN')['Tmed'].shift(i)
    Datos[f'Presion_atms_{i}d'] = Datos.groupby('ESTACIÓN')['Presion_atms'].shift(i)
    Datos[f'Vel_viento_{i}d'] = Datos.groupby('ESTACIÓN')['Vel_viento'].shift(i)
    Datos[f'Dir_viento_{i}d'] = Datos.groupby('ESTACIÓN')['Dir_viento'].shift(i) 

print(Datos)

In [ ]:
# Eliminar filas con valores vacíos debido a los shift's
Datos = Datos.dropna()   

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# SELECCIÓN FINAL DE LA BASE DE DATOS

In [ ]:
# Seleccionar las columnas que queremos en la base de datos final
columns = ['Latitud', 'Longitud', 'MES'] + [f'{var}_{i}d' for var in ['Lluvia', 'Humedad_R', 'Tmin', 'Tmax', 'Tmed', 'Vel_viento', 'Dir_viento', 'Presion_atms'] for i in range(1, 8)] + ['Anom_prec_mañana', 'Lluvia_Binaria', 'percentil'] + ['Media_prec_mañana_Clim']
DatosFinales = Datos[columns]

# Reiniciar el índice del DataFrame resultante
DatosFinales.reset_index(drop=True, inplace=True)

print(DatosFinales)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# EXPORTACIÓN DE LA NUEVA BASE DE DATOS

In [ ]:
# Guardar el DataFrame resultante en un nuevo archivo Excel
output_file_path = 'BaseDatosProcesada.xlsx'
DatosFinales.to_excel(output_file_path, index=False)